In [ ]:
# Setting up the analysis. Don't touch unless necessary
# https://www.cms.gov/medicare/payment/prospective-payment-systems/acute-inpatient-pps/ms-drg-classifications-and-software

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as mticker

AAA = pd.read_excel('AAA_wEverything_2023_12_08.xlsx')
AAA.set_index('EPIC',verify_integrity=True, inplace=True)

AAA['TOTAL_COSTS'] = AAA['PRO_COSTS'] + AAA['HOS_COSTS']
AAA['TOTAL_PAYMENTS'] = AAA['PRO_PAYMENT'] + AAA['HOS_PAYMENTS']
AAA['NET_INCOME'] = AAA['TOTAL_PAYMENTS'] - AAA['TOTAL_COSTS']

# Bill paid? yay or nay?
AAA_Outst = AAA[AAA['ANY_OUTSTANDING']==1]
AAA_NoOut = AAA[AAA['ANY_OUTSTANDING']==0]

# Medicare
Med_Only = AAA_NoOut[AAA_NoOut['PAYOR']=='MED']

#splitting by rupture status
rupture = Med_Only[Med_Only['Rupture_Flag']==1]
no_rupt = Med_Only[Med_Only['Rupture_Flag']==0]

def proc_numbers(df=AAA):
# Assign a unique number to each distinct procedure
    df['Procedure_Number'], uniques = pd.factorize(df['Surgery_Procedure'])
    procedure_counts = df['Surgery_Procedure'].value_counts().reset_index()
    procedure_counts.columns = ['Name', 'Count']
    procedure_counts['Procedure_Number'] = procedure_counts['Name'].map(uniques.get_loc)
    procedure_counts = procedure_counts[['Procedure_Number', 'Name', 'Count']]
    procedure_counts.set_index('Procedure_Number', inplace=True)
    print(procedure_counts)

def costs_graph(df=AAA, total=True, hos=True, pro=False):
    plt.figure(figsize=(14,8))
    if total:
        sns.histplot(df['TOTAL_COSTS'], color='red', kde=True, bins=100, label = 'Total Costs')
    if hos:
        sns.histplot(df['HOS_COSTS'], color='blue', kde=True, bins=100, label = 'Hospital Costs')
    if pro:
        sns.histplot(df['PRO_COSTS'], color='green', kde=True, bins=100, label = 'Professional Costs')
    plt.title('Distribution of Costs')
    plt.legend();

def payments_graph(df=AAA, total=True, hos=True, pro=False):
    plt.figure(figsize=(14,8))
    if total:
        sns.histplot(df['TOTAL_PAYMENTS'], color='red', kde=True, bins=100, label = 'Total Costs')
    if hos:
        sns.histplot(df['HOS_PAYMENTS'], color='blue', kde=True, bins=100, label = 'Hospital Costs')
    if pro:
        sns.histplot(df['PRO_PAYMENT'], color='green', kde=True, bins=100, label = 'Professional Costs')
    plt.title('Distribution of Payments')
    plt.legend();

# graphing profit and loss for each encounter
def NI_graph(df=AAA):
    plt.figure(figsize=(14,8))
    sns.histplot(df['NET_INCOME'], color='red', kde=True, bins=100, label = 'Total Costs')
    plt.title('Distribution of Profit/(Loss) Per Encounter');

def paymentsandcosts_graph(datfra=AAA, ctotal=True, chos=True, cpro=False, ptotal=True, phos=True, ppro=False):
    costs_graph(df=datfra, total=ctotal, hos=chos, pro=cpro)
    payments_graph(df=datfra, total=ptotal, hos=phos,pro=ppro)
    NI_graph(df=datfra);

def paymentsandcosts_stats(df=AAA):
    print("Median of payments is : $", round(np.median(df['TOTAL_PAYMENTS']),1))
    print("Median of costs is : $", round(np.median(df['TOTAL_COSTS']),1))
    print("Median of net income is : $", round(np.median(df['NET_INCOME']),1))
    print("STD of payments is : $",round(np.std(df['TOTAL_PAYMENTS']),1))
    print("STD of costs is : $",round(np.std(df['TOTAL_COSTS']),1))
    print("STD of net income is : $", round(np.std(df['NET_INCOME']),1))

def graph_metrics(metric='median'):
    # Set the positions and width for the bars
    positions = np.arange(3)  # Three positions for the three groups
    width = 0.2  # Width of each bar

    # Calculate means or medians for each dataframe
    if metric=='mean':
        values_func = np.mean
    elif metric=='std':
        values_func = np.std
    else:
        values_func = np.median

    values_AAA = [values_func(AAA['TOTAL_PAYMENTS']), values_func(AAA['TOTAL_COSTS']), values_func(AAA['NET_INCOME'])]
    values_AAA_Outst = [values_func(AAA_Outst['TOTAL_PAYMENTS']), values_func(AAA_Outst['TOTAL_COSTS']), values_func(AAA_Outst['NET_INCOME'])]
    values_AAA_NoOut = [values_func(AAA_NoOut['TOTAL_PAYMENTS']), values_func(AAA_NoOut['TOTAL_COSTS']), values_func(AAA_NoOut['NET_INCOME'])]

    # Plotting the bars
    fig, ax = plt.subplots(figsize=(11,3))

    # Create bars for each group
    ax.bar(positions - width, values_AAA, width, label='AAA')
    ax.bar(positions, values_AAA_Outst, width, label='Billed')
    ax.bar(positions + width, values_AAA_NoOut, width, label='Fin/Comp')

    # Adding labels to each bar
    for i in range(3):
        ax.text(i - width, values_AAA[i] + 1, round(values_AAA[i]/1000, 1), ha='center')
        ax.text(i, values_AAA_Outst[i] + 1, round(values_AAA_Outst[i]/1000, 1), ha='center')
        ax.text(i + width, values_AAA_NoOut[i] + 1, round(values_AAA_NoOut[i]/1000, 1), ha='center')

    ax.set_ylabel('Values')
    ax.set_title(f'Comparison of {metric} Values')
    ax.set_xticks(positions)
    ax.set_xticklabels(['TOTAL_PAYMENTS', 'TOTAL_COSTS', 'NET_INCOME'])
    ax.legend()

def plot_data_counts(size=(5,5),df1=AAA,df2=AAA_NoOut,df3=AAA_Outst):
    counts = [len(df1), len(df2), len(df3)]
    fig, ax = plt.subplots(figsize=size)
    ax.bar(['Full AAA', 'Final/Complete Only', 'Billed Only'], counts, color=['blue','green','black'])
    ax.set_title('Number of Patients')

def test_means(df1=AAA, df2=AAA_NoOut):
    for col in ['TOTAL_PAYMENTS', 'TOTAL_COSTS', 'NET_INCOME']:
        t_stat, p_value = stats.ttest_ind(df1[col], df2[col])
        same = (p_value >= 0.05)
        print(col, "P-value:", np.round(p_value,3))
        print("Are they statistically the same?",same,"\n")

def skew_kurt_costs(df=AAA):
    print("K^2 D'Agostino Test for Normalcy:\n")
    stat, pval = stats.normaltest(df['HOS_COSTS'])
    print("P-value for Hospital Costs: ", pval)

    stat, pval = stats.normaltest(df['PRO_COSTS'])
    print("P-value for Professional Costs: ", pval)

    stat, pval = stats.normaltest(df['TOTAL_COSTS'])
    print("P-value for Total Costs: ", pval)

    print("\n\nKolmogorov-Smirnov Test for Normalcy:\n")
    scaler = StandardScaler()

    hos_scaled = scaler.fit_transform(df['HOS_COSTS'].values.reshape(-1,1)).flatten()
    pro_scaled = scaler.fit_transform(df['PRO_COSTS'].values.reshape(-1,1)).flatten()
    tot_scaled = scaler.fit_transform(df['TOTAL_COSTS'].values.reshape(-1,1)).flatten()

    stat, pval = stats.kstest(hos_scaled, 'norm')
    print("P-value for Hospital Costs: ", pval)

    stat, pval = stats.kstest(pro_scaled, 'norm')
    print("P-value for Professional Costs: ", pval)

    stat, pval = stats.kstest(tot_scaled, 'norm')
    print("P-value for Total Costs: ", pval)

    a = df['HOS_COSTS'].kurtosis()
    b = df['HOS_COSTS'].skew()

    c = df['PRO_COSTS'].kurtosis()
    d = df['PRO_COSTS'].skew()

    e = df['TOTAL_COSTS'].kurtosis()
    f = df['TOTAL_COSTS'].skew()

    print("\n\nKurtosis and Skewness:\n")
    print(f"Hospital Costs, Kurtosis is {a:.2f} while skewness is {b:.2f}")
    print(f"Professional Costs, Kurtosis is {c:.2f} while skewness is {d:.2f}")
    print(f"Total Costs, Kurtosis is {e:.2f} while skewness is {f:.2f}")

def skew_kurt_payments(df=AAA):
    print("K^2 D'Agostino Test for Normalcy:\n")
    stat, pval = stats.normaltest(df['HOS_PAYMENTS'])
    print("P-value for Hospital Payments: ", pval)

    stat, pval = stats.normaltest(df['PRO_PAYMENT'])
    print("P-value for Professional Payments: ", pval)

    stat, pval = stats.normaltest(df['TOTAL_PAYMENTS'])
    print("P-value for Total Paymentss: ", pval)

    print("\n\nKolmogorov-Smirnov Test for Normalcy:\n")
    scaler = StandardScaler()

    hos_scaled = scaler.fit_transform(df['HOS_PAYMENTS'].values.reshape(-1,1)).flatten()
    pro_scaled = scaler.fit_transform(df['PRO_PAYMENT'].values.reshape(-1,1)).flatten()
    tot_scaled = scaler.fit_transform(df['TOTAL_PAYMENTS'].values.reshape(-1,1)).flatten()

    stat, pval = stats.kstest(hos_scaled, 'norm')
    print("P-value for Hospital Payments: ", pval)

    stat, pval = stats.kstest(pro_scaled, 'norm')
    print("P-value for Professional Payments: ", pval)

    stat, pval = stats.kstest(tot_scaled, 'norm')
    print("P-value for Total Payments: ", pval)

    a = df['HOS_PAYMENTS'].kurtosis()
    b = df['HOS_PAYMENTS'].skew()

    c = df['PRO_PAYMENT'].kurtosis()
    d = df['PRO_PAYMENT'].skew()

    e = df['TOTAL_PAYMENTS'].kurtosis()
    f = df['TOTAL_PAYMENTS'].skew()

    print("\n\nKurtosis and Skewness:\n")
    print(f"Hospital Payments, Kurtosis is {a:.2f} while skewness is {b:.2f}")
    print(f"Professional Payments, Kurtosis is {c:.2f} while skewness is {d:.2f}")
    print(f"Total Payments, Kurtosis is {e:.2f} while skewness is {f:.2f}")

def skew_kurt_income(df=AAA):
    stat, pval = stats.normaltest(df['NET_INCOME'])
    print("K^2 D'Agostino Test for Normalcy P-value for Net Income: ", pval)

    scaler = StandardScaler()
    tot_scaled = scaler.fit_transform(df['NET_INCOME'].values.reshape(-1,1)).flatten()
    stat, pval = stats.kstest(tot_scaled, 'norm')
    print("Kolmogorov-Smirnov Test for Normalcy P-value for Net Income: ", pval)

    e = df['NET_INCOME'].kurtosis()
    f = df['NET_INCOME'].skew()
    print(f"Total Net Income, Kurtosis is {e:.2f} while skewness is {f:.2f}")

def full_skew_kurt(datfra=AAA):
    print("INFORMATION ON COSTS")
    print("*********************************************************")
    skew_kurt_costs(df=datfra)
    print("*********************************************************\n\n")

    print("INFORMATION ON PAYMENTS")
    print("*********************************************************")
    skew_kurt_payments(df=datfra)
    print("*********************************************************\n\n")

    print("INFORMATION ON INCOME")
    print("*********************************************************")
    skew_kurt_income(df=datfra)
    print("*********************************************************")

def column_graph(col, df=AAA, metric='median', min_count=10):
    temp_counts = df[col].value_counts()
    count_fil = temp_counts[temp_counts > min_count].index.tolist()
    df_filt = df[df[col].isin(count_fil)]

    temp_counts.plot(kind='bar', figsize=(12,8))
    plt.title("Counts by "+col)
    plt.ylabel("Counts")
    plt.xlabel('');

    agg_func = getattr(df_filt.groupby(col), metric)
    temp_groups = agg_func(numeric_only=True) if metric in ['mean', 'median', 'sum', 'std'] else agg_func()

    #temp_groups = df_filt.groupby(col).median(numeric_only=True)
    ax = temp_groups[['HOS_COSTS', 'PRO_COSTS', 'TOTAL_COSTS']].plot(kind='bar', figsize=(12,8))
    for p in ax.patches:
        value = f'${p.get_height():,.0f}'  # Formats the number as a dollar amount
        ax.annotate(value, (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
    plt.title("Costs by "+col)
    plt.ylabel('Costs '+metric)
    plt.xlabel('');

    ax = temp_groups[['HOS_PAYMENTS', 'PRO_PAYMENT', 'TOTAL_PAYMENTS']].plot(kind='bar', figsize=(12,8))
    for p in ax.patches:
        value = f'${p.get_height():,.0f}'  # Formats the number as a dollar amount
        ax.annotate(value, (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
    plt.title("Payments by "+col)
    plt.ylabel("Payments "+metric)
    plt.xlabel('');

    ax = temp_groups[['NET_INCOME']].plot(kind='bar', figsize=(12,8))
    for p in ax.patches:
        value = f'${p.get_height():,.0f}'  # Formats the number as a dollar amount
        ax.annotate(value, (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')
    plt.title("Profit / (Loss) by "+col)
    plt.ylabel(metric+" Profit / (Loss)")
    plt.xlabel('');

    if col == "Procedure_Number":
        print(procedure_counts)

# returns a df that filters out stays longer that specified value of days
def trim_by_LOS(days, df=AAA):
    return df[df['LOS']<=days]

def los_scatter(col_name, df=AAA):
    assert col_name in df.columns, "Please choose a column from the current dataframe"
    plt.figure(figsize=(14,8))
    sns.scatterplot(x='LOS',y=col_name,data=df)

def los_reg(col_name, df=AAA):
    assert col_name in df.columns, "Please choose a column from the current dataframe"
    X = df['LOS']
    y = df[col_name]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    print("This is a regression of Length of Stay (LOS) on the variable",col_name,"\n\n\n")
    print(model.summary())

def LOS_graph(df=AAA):
    plt.figure(figsize=(14,8))
    sns.histplot(df['LOS'], color='red', kde=True, bins=25)
    plt.title('Length of Stay');

def los_analysis(select_column, data=AAA):
    los_scatter(select_column,df=data)
    los_reg(select_column,df=data)
    LOS_graph(df=data)

def scans2find(r=0.25, s=0.50, scancost=400):

# Function to format y-axis as dollars
    def dollars(x, pos):
        return f'${x:,.0f}'

    p_values = np.linspace(0.02, 0.14, 100) #positivity rate

    n_positv = 1 / p_values
    n_ruptur = 1 / (p_values * r)
    n_emergs = 1 / (p_values * r * s)

    c_positv = n_positv * scancost
    c_ruptur = n_ruptur * scancost
    c_emergs = n_emergs * scancost

# Plot for quantities (n_positiv, n_ruptur, n_emergs)

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,6))

    ax1.plot(p_values * 100, n_positv, label='A Positive')
    ax1.plot(p_values * 100, n_ruptur, label='A Rupture')
    ax1.plot(p_values * 100, n_emergs, label='An Emergency Surgery')
    ax1.set_xlabel('p (%)')
    ax1.set_ylabel('Number of Scans')
    ax1.set_title('Number of Scans Needed for:')
    ax1.legend()

    ax2.plot(p_values * 100, c_positv, label='A Positive')
    ax2.plot(p_values * 100, c_ruptur, label='A Rupture')
    ax2.plot(p_values * 100, c_emergs, label='An Emergency Surgery')
    #plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(dollars))
    ax2.yaxis.set_major_formatter(mticker.FuncFormatter(dollars))
    ax2.set_xlabel('p (%)')
    ax2.set_ylabel('Total Cost ($)')
    ax2.set_title('Total Scanning Cost For Identifying/Preventing:')
    ax2.legend()

In [ ]:
class Cohort:
    """
    Defines a cohort of individuals flagged for AAA medial screening.
    """
    def __init__(self, scans, positives, rupture_rate=0.25, death_b4_treatment=0.50, emerg_surgery_success_rate=0.771,
                 total_cohort_size=900, scan_cost=400, proph_surgery_cost=31412, proph_surgery_payment=33229,
                 emerg_surgery_cost=38800, emerg_surgery_payment=39187, use_default_positivity_rate=True):
        """
        Initializes the Cohort with given parameters.

        Default probabilities and success rates from literature. Prices are based on median values from EHR.
        """

        # Assertions for input validation
        assert positives <= scans <= total_cohort_size, "Invalid input: positives <= scans <= total_cohort_size must be true"
        assert 0 <= rupture_rate <= 1.00, "Invalid rupture_rate: must be between 0 and 1"
        assert 0 <= death_b4_treatment <= 1.00, "Invalid death_b4_treatment rate: must be between 0 and 1"
        assert 0 <= emerg_surgery_success_rate <= 1.00, "Invalid emerg_surgery_success_rate: must be between 0 and 1"

        self.positives = positives
        self.scans = scans

        # from research
        self.rupture_rate = rupture_rate
        self.death_b4_treatment = death_b4_treatment
        self.emerg_surgery_success_rate = emerg_surgery_success_rate

        # program definition of size and program assumption on positivity
        self.total_cohort_size = total_cohort_size
        self.use_default_positivity_rate = use_default_positivity_rate

        # representative cost assumptions
        self.scan_cost = scan_cost
        self.proph_surgery_cost = proph_surgery_cost
        self.emerg_surgery_cost = emerg_surgery_cost

        # representative payments assumptions
        self.proph_surgery_payment = proph_surgery_payment
        self.emerg_surgery_payment = emerg_surgery_payment

    def scanning_costs(self):
        """
        Calculate and return the total cost of scanning.

        :return: Total cost of scanning.
        """
        total = self.scans * self.scan_cost
        return total

    def scanned_positivity_rate(self):
        """
        Calculate and return the positivity rate of those scanned.

        :return: Positivity rate.
        """
        rate = self.positives / self.scans
        return rate

    def _get_positivity_rate(self):
        """
        Get the positivity rate, either the default or calculated.

        :return: Positivity rate.
        """
        if self.use_default_positivity_rate:
            return 27/416 #reflects positivity rate of the program so far
        else:
            return self.scanned_positivity_rate()

    def proph_surgeries(self):
        """
        Calculate and return the number of prophylactic surgeries needed based on positivity.

        :return: Number of prophylactic surgeries.
        """
        surgeries = self.positives * self.rupture_rate
        return surgeries

    def proph_surgery_costs(self):
        """
        Calculate and return the total cost for prophylactic surgeries.

        :return: Total cost for prophylactic surgeries.
        """
        total = self.proph_surgeries() * self.proph_surgery_cost
        return total

    def proph_surgery_payments(self):
        """
        Calculate and return the total cost for prophylactic surgeries.

        :return: Total cost for prophylactic surgeries.
        """
        total = self.proph_surgeries() * self.proph_surgery_payment
        return total

    def untreated_deaths(self):
        """
        Calculate and return the number of untreated deaths in the cohort.

        :return: Number of untreated deaths.
        """
        positivity_rate = self._get_positivity_rate()

        deaths = (self.total_cohort_size - self.scans) * self.rupture_rate * self.death_b4_treatment * positivity_rate
        return deaths

    def emerg_surgeries(self):
        """
        Calculate and return the number of emergency surgeries needed.

        :return: Number of emergency surgeries.
        """
        positivity_rate = self._get_positivity_rate()

        surgeries = (self.total_cohort_size - self.scans) * self.rupture_rate * (1 - self.death_b4_treatment) * positivity_rate
        return surgeries

    def emerg_surgery_costs(self):
        """
        Calculate and return the total cost for emergency surgeries.

        :return: Total cost for emergency surgeries.
        """
        total = self.emerg_surgeries() * self.emerg_surgery_cost
        return total

    def emerg_surgery_payments(self):
        """
        Calculate and return the total cost for emergency surgeries.

        :return: Total cost for emergency surgeries.
        """
        total = self.emerg_surgeries() * self.emerg_surgery_payment
        return total

    def emerg_surgery_deaths(self):
        """
        Calculate and return the number of deaths resulting from emergency surgeries.

        :return: Number of deaths from emergency surgeries.
        """
        deaths = self.emerg_surgeries() * (1 - self.emerg_surgery_success_rate)
        return deaths

    def total_cost(self):
        """
        Calculate and return the total cost for all surgeries and scans.

        :return: Total cost for surgeries and scans.
        """
        total = self.scanning_costs() + self.proph_surgery_costs() + self.emerg_surgery_costs()
        return total

    def total_payments(self):
        """
        Calculate and return the total cost for all surgeries and scans.

        :return: Total cost for surgeries and scans.
        """
        total = self.scanning_costs() + self.proph_surgery_payments() + self.emerg_surgery_payments()
        return total

    def total_income(self):
        """
        Calculate and return the total cost for all surgeries and scans.

        :return: Total cost for surgeries and scans.
        """
        total = self.total_payments() - self.total_cost()
        return total

    def total_deaths(self):
        """
        Calculate and return the total number of deaths in the cohort.

        :return: Total number of deaths.
        """
        total = self.untreated_deaths() + self.emerg_surgery_deaths()
        return total

    def full_results(self,printout=True):
        """
        Aggregate and format results from various methods to display a comprehensive overview of the cohort's outcomes.

        :return: None, but prints the results.
        """
        methods_to_call = ['emerg_surgeries', 'emerg_surgery_costs', 'emerg_surgery_payments', 'emerg_surgery_deaths',
                           'scanned_positivity_rate', 'proph_surgeries', 'proph_surgery_costs', 'proph_surgery_payments',
                           'scanning_costs', 'total_cost', 'total_payments', 'total_income','total_deaths',
                           'untreated_deaths']

        results = {}

        for method_name in methods_to_call:
            method = getattr(self, method_name)
            results[method_name] = method()

        if printout==False:
            return results

        else:

        # Formatting the values
            results['emerg_surgeries'] = round(results['emerg_surgeries'], 1)
            results['emerg_surgery_deaths'] = round(results['emerg_surgery_deaths'], 1)
            results['proph_surgeries'] = round(results['proph_surgeries'], 1)
            results['total_deaths'] = round(results['total_deaths'], 1)
            results['untreated_deaths'] = round(results['untreated_deaths'], 1)

            results['emerg_surgery_costs'] = "${:,.0f}".format(results['emerg_surgery_costs'])
            results['proph_surgery_costs'] = "${:,.0f}".format(results['proph_surgery_costs'])
            results['emerg_surgery_payments'] = "${:,.0f}".format(results['emerg_surgery_payments'])
            results['proph_surgery_payments'] = "${:,.0f}".format(results['proph_surgery_payments'])

            results['scanning_costs'] = "${:,.0f}".format(results['scanning_costs'])
            results['total_cost'] = "${:,.0f}".format(results['total_cost'])
            results['total_payments'] = "${:,.0f}".format(results['total_payments'])
            results['total_income'] = "${:,.0f}".format(results['total_income'])

            results['scanned_positivity_rate'] = "{:.1%}".format(results['scanned_positivity_rate'])

            for key, value in results.items():
                print(f"{key}: {value}")
                print("-------------------------------")

In [ ]:
# CHOOSE MONTHS code

#!/usr/bin/env python3
import sys
from calendar import monthrange
from datetime import datetime, timedelta

def parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%m/%Y")
    except ValueError:
        raise ValueError(f"Date format should be MM/YYYY. Invalid date: {date_str}")

def get_last_days_of_months(start_date, end_date):
    last_days = []
    current_date = start_date

    while current_date <= end_date:
        year, month = current_date.year, current_date.month
        last_day = monthrange(year, month)[1]
        last_days.append(f"{year}-{month:02d}-{last_day:02d}")
        if month == 12:
            current_date = current_date.replace(year=year+1, month=1)
        else:
            current_date = current_date.replace(month=month+1)
    return last_days

start_date_str = "04/2020"
end_date_str = "07/2022"

if len(sys.argv) == 3:
    start_date_str = sys.argv[1]
    end_date_str = sys.argv[2]
else:
    print("uh oh")

try:
    start_date = parse_date(start_date_str)
    end_date = parse_date(end_date_str)
except ValueError as e:
    print(e)
    sys.exit(1)

if start_date > end_date:
    print("The end date must be after the start date.")
    sys.exit(1)

last_days = get_last_days_of_months(start_date, end_date)
print(last_days)
print(sys.argv)

In [ ]:
#combine months
import pandas as pd
import os

file_paths = os.listdir("output")
file_paths = [os.path.join("output", f) for f in file_paths if f.endswith(".txt")]
file_paths
#file_paths = ["output/AAA-Flag_output_2022-02-28.txt", "output/AAA-Flag_output_2022-03-31.txt"]

def combine_top_150(file_paths):
    combined_output = pd.DataFrame()
    ids_so_far = []
    for file_path in file_paths:
        current_file = pd.read_csv(file_path, sep="\t", usecols=['ID', 'CorrelationId', 'Score'])
        current_file = current_file[~current_file['ID'].isin(ids_so_far)]
        current_top_150 = current_file.nlargest(150, 'Score')
        ids_so_far = list(set(list(current_top_150.ID) + ids_so_far))
        current_top_150['date'] = file_path[-14:-4]
        combined_output = pd.concat([combined_output, current_top_150], ignore_index=True)
    return combined_output

total_output = combine_top_150(file_paths)
start_date = file_paths[0][-14:-4]
end_date = file_paths[-1][-14:-4]
filename = f"Combined_Output_{start_date}_{end_date}.txt"
total_output.to_csv(filename, sep="\t", index=False)

In [16]:
# combine historical

import pandas as pd
import os

@njit
def combine_distinct(file_paths):
    combined_output = pd.DataFrame()
    ids_so_far = []
    for file_path in file_paths:
        current_file = pd.read_csv(file_path, sep="\t", usecols=['ID', 'CorrelationId', 'Score'])
        current_file = current_file[~current_file['ID'].isin(ids_so_far)]
        current_top_150 = current_file.nlargest(150, 'Score')
        for bob in list(current_top_150.ID):
            ids_so_far.append(bob)
        current_file['date'] = file_path[-14:-4]
        combined_output = pd.concat([combined_output, current_file], ignore_index=True)
    return combined_output

file_paths = os.listdir('Pre--Eldan')
file_paths = [os.path.join("Pre--Eldan", f) for f in file_paths if f.endswith(".txt")]

total_output = combine_distinct(file_paths)
total_output.to_csv('PreEldan_AllHistorical', sep="\t", index=False)

file_paths = os.listdir('Post-Eldan')
file_paths = [os.path.join("Post-Eldan", f) for f in file_paths if f.endswith(".txt")]

total_output = combine_distinct(file_paths)
total_output.to_csv('PostEldan_AllHistorical', sep="\t", index=False)

In [19]:
#dis.dis(combine_distinct)
#!pip install numba

import dis
from numba import njit

combine_distinct.inspect_asm()

{}

In [ ]:
# Step 1: Import Libraries
import pandas as pd
import os

# Step 2: Define function -be careful with names as ID can be DeID or PatID depending on contest
def combine_distinct(file_paths):
    all_eligible = pd.DataFrame()
    ids_so_far = [] # id's of the highest 150 each month so far
    top = pd.DataFrame()
    for file_path in file_paths:
        current_file = pd.read_csv(file_path, sep="\t", usecols=['ID', 'CorrelationId', 'Score'])
        current_file = current_file[~current_file['ID'].isin(ids_so_far)]
        current_file['date'] = file_path[-14:-4]
        current_top_150 = current_file.nlargest(150, 'Score')
        top = pd.concat([top, current_top_150], ignore_index=True)
        for patient in list(current_top_150.ID):
            ids_so_far.append(patient)
        all_eligible = pd.concat([all_eligible, current_file], ignore_index=True)
    return top, all_eligible, ids_so_far

# Step 3: Get file paths and run function
file_paths = os.listdir()
file_paths = [f for f in file_paths if f.endswith(".txt")]
top, all_eligible, ids = combine_distinct(file_paths)

# Step 4: Gettng the Pat_id back from the anonymized version

# read in deid file, confirm one unique de-identified ID per form, then filter to ONLY
# those rows that correspond to one of our top ID's and merge with the top file
idfile = pd.read_csv('DEID_PAT.csv',dtype={'ID':'Int64'},usecols=['ID','pat_id'])
assert len(idfile['ID']) == idfile['ID'].nunique()
#total_output = pd.merge(total_output, current_file, how='left', on='ID')
idfile = idfile[idfile['ID'].isin(ids)]
top_with_ids = pd.merge(top,idfile,on='ID')

# Step 5: Export Files for Use
total_output.to_csv('EightMonth_AllEligible', sep="\t", index=False)
top_with_ids.to_csv('EightMonth_Top150PerMonth', sep="\t", index=False)

In [ ]:
# sending emails
import pandas as pd
import smtplib
from email.message import EmailMessage

def send_email(subject, body, to_email, smtp_server, smtp_port, smtp_user, smtp_password):
    msg = EmailMessage()
    msg.set_content(body)
    msg['Subject'] = subject
    msg['From'] = smtp_user
    msg['To'] = to_email

    # Set up the SMTP server and send the email
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()  # This secures the connection
    server.login(smtp_user, smtp_password)  # Login with the provided credentials
    server.send_message(msg)
    server.quit()

def main(csv_path, txt_path, smtp_server, smtp_port):
    # Prompt the user for their email and password
    smtp_user = input("Enter your Outlook email address: ")
    smtp_password = input("Enter your Outlook password: ")

    # Read the call script text from the TXT file
    with open(txt_path, 'r') as file:
        template_text = file.read()

    # Read the CSV file with pandas
    data = pd.read_csv(csv_path)

    # Loop through each row in the CSV and send emails where the 'INCLUDE' column is 'YES'
    for index, row in data.iterrows():
        if row['INCLUDE'] == 'YES':
            personalized_text = template_text.replace('[INSERT LAST NAME]', row['NAME'])
            send_email("AAA Screening Reminder", personalized_text, row['EMAIL'], smtp_server, smtp_port, smtp_user, smtp_password)

if __name__ == "__main__":
    # Replace the placeholders below with the actual SMTP settings provided by  IT department
    smtp_server = 'your_smtp_server'  # e.g., 'smtp.geisinger.org'
    smtp_port = your_smtp_port  # e.g., 587 or 465

    csv_file_path = 'AAA_Names.csv'
    txt_file_path = 'AAA_Call_Script.txt'

    main(csv_file_path, txt_file_path, smtp_server, smtp_port)
